머신러닝 종류 : 교사, 비교사, 강화

모델
- 딥러닝   : 선형회구, 논리회구
- 선형회귀 : Y = WX + b, X(x1, x2, xn) : 실수예측
- 논리회귀 : Y = sigmoid(WX+b) = [0, 1] 확률 : 2진 분류
- 다중논리회귀 : Y = softmax(W_k * X + b_k) : k개 분류
    ex) 날씨 k =3(맑음/흐림/비), Y(0, 1, 2)
        X(온도, 습도, 바람) = (x1, x2, x3)
        맑을 가능성 = W_k1 * X + b_k1 = W_맑음 * X + b_맑음
        습할 가능성 = W_k2 * X + b_k2 = W_습도 * X + b_습도
        비올 가능성 = W_k3 * X + b_k3 = W_비 * X + b_비
        
        가능성(score, logit) = (맑을 가능성, 습할 가능성, 비올 가능성) = (10, 80, 40)
        softmax(10, 80, 40) = (0.02, 0.9, 0.08) -- 확률로 변경
        
        Y 원핫 인코딩 Y=1 => (0, 1, 0)
        
파라미터       : W와 b 같이 모델이 fit하는 값
하이퍼파라미터 : 학습률과 정규화 같이 사람이 결정하는 값

머신러닝 라이브러리 : sklearn(하이퍼파라미터 자동설정) <-> tenserflow(우리가 직접 셋팅)

# 멀티 클래스 분류

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
from sklearn.datasets import load_iris

In [5]:
# 3종류 붓꽃 분류
# X : 꽃의 길이 너비, 꽃받침의 길이 너미 4개
X, y = load_iris(return_X_y=True)
X.shape, y.shape, min(y), max(y)

((150, 4), (150,), 0, 2)

In [6]:
from sklearn.linear_model import LogisticRegression

In [11]:
model = LogisticRegression().fit(X, y)
print('accuray', model.score(X, y))
print('w', model.coef_, 'b', model.intercept_)
# Y = W_k * X _ b_k
# W : (3, 4), b:(3) => 클래스 종류가 3개이고, 4개의 특성이 있다.

accuray 0.96
w [[ 0.41021713  1.46416217 -2.26003266 -1.02103509]
 [ 0.4275087  -1.61211605  0.5758173  -1.40617325]
 [-1.70751526 -1.53427768  2.47096755  2.55537041]] b [ 0.26421853  1.09392467 -1.21470917]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [15]:
type(X), type(y), np.max(X), np.min(X), X.shape, y.shape

(numpy.ndarray, numpy.ndarray, 7.9, 0.1, (150, 4), (150,))

In [16]:
from sklearn.preprocessing import OneHotEncoder

In [22]:
y_2d = y.reshape([-1, 1])
y_hot = OneHotEncoder().fit_transform(y_2d)
y_hot = y_hot.toarray()
x = X.astype(np.float32)
x.shape, y_hot.shape

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


((150, 4), (150, 3))

In [18]:
w = tf.Variable(tf.random_normal(shape=[4, 3]))
b = tf.Variable(tf.random_normal(shape=[3]))
xn = x / tf.reduce_max(x)
z = tf.matmul(xn, w) + b
h = tf.nn.softmax(z)
h = tf.clip_by_value(h, 1e-8, 1- 1e-8) # log(0)이 되지 않도록 안전장치
cost = -tf.reduce_mean(y_hot * tf.log(h) + (1-y_hot) * tf.log(1-h))
predict_class = tf.math.argmax(h, axis=1, output_type=tf.int32) # [0, 2]
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict_class, y), tf.float32))
train = tf.train.GradientDescentOptimizer(0.5).minimize(cost)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

W0909 09:52:57.951887  7416 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [20]:
for i in range(1000):
    _, c, a = sess.run([train, cost, accuracy])
    if i%100==0:
        print(i, c, a)

0 0.25077334 0.9533333
100 0.24228756 0.96
200 0.23459691 0.96
300 0.22757462 0.96666664
400 0.22112331 0.96666664
500 0.21516612 0.96666664
600 0.2096412 0.96666664
700 0.20449778 0.96666664
800 0.19969349 0.96666664
900 0.19519275 0.96666664
